In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [18]:
df = pd.read_csv(r'C:\Users\greym\Desktop\dataset for 446\energydata.csv')
df.head()

,REF_DATE,GEO,DGUID,Type of dwelling,Energy type,Energy consumption,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2011,Canada,2016A000011124,Single-detached,"Total, all energy types",Gigajoules,Gigajoules,143,units,0,v101904406,1.1.1.1,944421514.0,NaN,NaN,NaN,0
1,2011,Canada,2016A000011124,Single-detached,"Total, all energy types",Gigajoules per household,Gigajoules,143,units,0,v101904407,1.1.1.2,115.3,NaN,NaN,NaN,1
2,2011,Canada,2016A000011124,Single-detached,"Total, all energy types",Number of households,Number,223,units,0,v101904408,1.1.1.4,8191007.0,NaN,NaN,NaN,0
3,2011,Canada,2016A000011124,Single-detached,Electricity,Gigajoules,Gigajoules,143,units,0,v101904409,1.1.2.1,396962347.0,NaN,NaN,NaN,0
4,2011,Canada,2016A000011124,Single-detached,Electricity,Gigajoules per household,Gigajoules,143,units,0,v101904410,1.1.2.2,48.5,NaN,NaN,NaN,1


In [19]:
# drop uncessary data
df.drop(['DGUID','UOM_ID','SCALAR_ID','Type of dwelling','SCALAR_FACTOR','VECTOR','COORDINATE','STATUS','SYMBOL', 'DECIMALS','TERMINATED'],inplace=True, axis=1)
df.head()

,REF_DATE,GEO,Energy type,Energy consumption,UOM,VALUE
0,2011,Canada,"Total, all energy types",Gigajoules,Gigajoules,944421514.0
1,2011,Canada,"Total, all energy types",Gigajoules per household,Gigajoules,115.3
2,2011,Canada,"Total, all energy types",Number of households,Number,8191007.0
3,2011,Canada,Electricity,Gigajoules,Gigajoules,396962347.0
4,2011,Canada,Electricity,Gigajoules per household,Gigajoules,48.5


In [20]:
#drop more data
newdf = df.dropna()
newdf.head()

,REF_DATE,GEO,Energy type,Energy consumption,UOM,VALUE
0,2011,Canada,"Total, all energy types",Gigajoules,Gigajoules,944421514.0
1,2011,Canada,"Total, all energy types",Gigajoules per household,Gigajoules,115.3
2,2011,Canada,"Total, all energy types",Number of households,Number,8191007.0
3,2011,Canada,Electricity,Gigajoules,Gigajoules,396962347.0
4,2011,Canada,Electricity,Gigajoules per household,Gigajoules,48.5


In [21]:
# drop all data that does not have the energy type of electricity and has a location of canada
newdf.drop(newdf[newdf['GEO'] == 'Canada'].index, inplace = True)
newdf.drop(newdf[newdf['Energy type'] != 'Electricity'].index, inplace = True)
newdf.head()

C:\Users\greym\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,REF_DATE,GEO,Energy type,Energy consumption,UOM,VALUE
108,2011,Newfoundland and Labrador,Electricity,Gigajoules,Gigajoules,11934482.0
109,2011,Newfoundland and Labrador,Electricity,Gigajoules per household,Gigajoules,72.7
110,2011,Newfoundland and Labrador,Electricity,Proportion of total energy,Percent,77.6
111,2011,Newfoundland and Labrador,Electricity,Number of households,Number,164067.0
213,2011,Prince Edward Island,Electricity,Gigajoules,Gigajoules,1147772.0


In [22]:
# Select columns of electricity consumption that is gigajoules
energy=newdf.loc[newdf['Energy consumption'] == 'Gigajoules']
y = energy.iloc[:, [5]]
y.head()

,VALUE
108,11934482.0
213,1147772.0
318,9704911.0
363,685874.0
378,1429845.0


In [23]:
# Select column that is giga per household
household=newdf.loc[newdf['Energy consumption'] == 'Gigajoules per household']
z = household.iloc[:, [5]]
z.head()

,VALUE
109,72.7
214,26.5
319,36.0
364,31.0
379,23.6


In [24]:
# Select columns of proportion of total energy that is gigajoules
proportion=newdf.loc[newdf['Energy consumption'] == 'Proportion of total energy']
x2 = proportion.iloc[:, [5]]
x2.head()

,VALUE
110,77.6
215,28.6
320,47.9
365,56.2
380,35.1


In [25]:
# Select columns of proportion of number of households that is gigajoules
number=newdf.loc[newdf['Energy consumption'] == 'Number of households']
x3 = number.iloc[:, [5]]
x3.head()

,VALUE
111,164067.0
216,43300.0
321,269296.0
366,22136.0
381,60523.0


In [26]:
#drop the 12 extra rows 
perhousehold = z.drop([3349,3304,3109,2584,2299,1969,1129,1084,1039,814,2149,4039])
proportion = x2.drop([815,1040,1085,2150,2300,2585,3110,3305])
number= x3.drop([1086,1971,2301,2586,3111,3351])

In [27]:
#Inserting all 3 dataframes
perhousehold = perhousehold.reset_index(drop = True)
proportion = proportion.reset_index(drop =True)
number = number.reset_index(drop = True)
X = pd.concat([perhousehold,number],axis=1,keys=['Perhousehold', 'number'])
X

,Perhousehold,number
,VALUE,VALUE
0,72.7,164067.0
1,26.5,43300.0
2,36.0,269296.0
3,31.0,22136.0
4,23.6,60523.0
...,...,...
118,31.7,141375.0
119,38.8,54159.0
120,16.3,347275.0


In [28]:
# dependent and independent variables
X1 = [X]
Y = y.reset_index(drop = True)
Y

,VALUE
0,11934482.0
1,1147772.0
2,9704911.0
3,685874.0
4,1429845.0
...,...
118,4480597.0
119,2099854.0
120,5651427.0
121,2979272.0


In [44]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=0)
# Transform the input data to include polynomial terms up to degree 2

poly = PolynomialFeatures(degree = 2)
X_train_poly = poly.fit_transform(x_train)
X_test_poly = poly.transform(x_test)

# Fit the train model using linear regression
model = LinearRegression().fit(X_train_poly, y_train)

y_pred = model.predict(X_test_poly)
y_pred


array([[ 1.29446532e+07],
       [ 6.52937816e+07],
       [ 1.74424781e+07],
       [ 4.62016104e+06],
       [ 1.07178532e+07],
       [ 1.15115624e+07],
       [ 2.72137231e+06],
       [ 1.32381801e+08],
       [ 1.99755294e+07],
       [ 2.37855654e+06],
       [ 2.94899948e+06],
       [ 2.43152454e+07],
       [ 1.66281684e+06],
       [ 1.45391169e+08],
       [ 5.92172572e+06],
       [ 8.17032004e+06],
       [ 1.13154949e+07],
       [ 7.43036700e+06],
       [-6.85235521e+05],
       [ 2.55354521e+07],
       [ 1.92375443e+07],
       [ 2.18339243e+07],
       [ 7.56691627e+06],
       [-1.14920642e+06],
       [ 8.21290027e+06],
       [ 3.09037814e+07],
       [ 1.12121476e+08],
       [ 2.21300401e+06],
       [ 1.07946531e+08],
       [ 9.94372866e+06],
       [ 3.82944111e+07]])

In [43]:
mse = mean_squared_error(y_test,y_pred)  # calculate MSE
mae = mean_absolute_error(y_test, y_pred)  # calculate MAE
r2=r2_score(y_test,y_pred)# calculate R2

print("MAE:", mae)
print("MSE:", mse)
print("R2:", r2)

MAE: 6555100.627638567
MSE: 164818238851229.03
R2: 0.9045915736626329
